<h1>Inhaltsverzeichnis<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#C.-Portfolio-Management" data-toc-modified-id="C.-Portfolio-Management-1">C. Portfolio Management</a></span></li><li><span><a href="#C.3-Eine-erste-Handelsstrategie:-Low-Risk" data-toc-modified-id="C.3-Eine-erste-Handelsstrategie:-Low-Risk-2">C.3 Eine erste Handelsstrategie: Low Risk</a></span></li></ul></div>

# C. Portfolio Management
# C.3 Eine erste Handelsstrategie: Low Risk
Bei der Konstruktion jeder regelgebundenen, quantitativen Handelsstrategie sind die folgenden drei Aspekte von zentraler Bedeutung:
- Was ist die mit der Festlegung der Portfoliogewichte zu optimierende **Zielfunktion (Target Function)**? Mögliche Zielfunktionen können eindimensional sein, d.h., nur auf die Minimierung von Risiko oder die Maximierung von Rendite abstellen, oder eine Kombination beider Kriterien beinhalten (z.B. Maximierung der Sharpe-Ratio);
- Zur Bestimmung der unbekannten Parameter der Zielfunktion werden in der Regel historische Kurszeitreihen verwendet. Hier stellt sich die Frage nach der **optimalen Länge der Zeitreihen (Window Length)**. D.h., wie alt dürfen die Daten maximal sein, um noch als repräsentativ für die Zukunft zu gelten? Obwohl eine größere Anzahl an Datenpunkten ceteris paribus eine genauere Schätzung erlaubt, können Strukturbrüche in den Daten Parameterschätzungen verzerren.
- Die eine Zielfunktion optimierenden Portfoliogewichte verändern sich dynamisch mit den Daten die in die Zielfunktion eingehen. D.h., Gewichte sind immer nur in Bezug auf die gerade vorliegenden Daten optimal. Zudem verändern sich die einmal optimierten Portfoliogewichte mit den Kursveränderungen der im Portfolio enthaltenen Aktien. Es stellt sich daher die Frage nach der **optimalen Halteperiode (Rebalancing Frequency)** eines Portfolios, d.h., nach welchem Zeitraum sollten die aktuellen Portfoliogewichte wieder an die optimalen Gewichte, gegeben die dann aktuellen Datenzeitreihen, angepasst werden.  

In diesem Kapitel werden die beiden letzten Aspekte im Rahmen einer einfachen, aber in der Praxis weit verbreiteten Strategie betrachtet. Die Zielfunktion dieser Strategie ist eindimensional: das Portfoliorisiko, gemessen über die Varianz der Portfoliorendite, soll minimiert werden. Die erwartete Rendite geht nicht in die Zielfunktion ein. Solche Strategien nennt man allgemein **Low Risk Strategien.** 

Im nachfolgenden Code der Fallstudie greifen wir auf die im Notebook C.2 eingeführte Funktion `min_variance` zurück, um die Gewichte des varianzminimalen (oder vola-minimalen) Portfolios mit Hilfe von `scipy.optimize` zu bestimmen.   

Hinsichtlich der Länge der verwendeten Datenhistorie werden wir einen Ansatz mit rollierenden Zeitfenstern implementieren. Die Halteperiode ist ein einzustellender Parameter (Argument) in der Funktion `strat_sr`, die die Strategie implementiert. 

Beginnen wir wie üblich zunächst mit dem Laden der notwendigen Pakete.

In [1]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import scipy.optimize as sco
from datetime import datetime

#!pip install yfinance
import yfinance as yf

Laden der Kurshistorie für unser 5 Aktien-Universum.

In [2]:
#download stock portfolio data
yf.pdr_override()
tickers = ['AAPL', 'AMZN', 'GOOG', 'MSFT', 'NFLX']
start = datetime(2010, 1, 1)
end = datetime(2018, 12, 31)

df = pd.DataFrame([web.get_data_yahoo(ticker, start, end)['Adj Close'] for ticker in tickers]).T
df.columns = tickers

df = df.asfreq('B').fillna(method='pad')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
df.index

DatetimeIndex(['2010-01-04', '2010-01-05', '2010-01-06', '2010-01-07',
               '2010-01-08', '2010-01-11', '2010-01-12', '2010-01-13',
               '2010-01-14', '2010-01-15',
               ...
               '2018-12-17', '2018-12-18', '2018-12-19', '2018-12-20',
               '2018-12-21', '2018-12-24', '2018-12-25', '2018-12-26',
               '2018-12-27', '2018-12-28'],
              dtype='datetime64[ns]', name='Date', length=2345, freq='B')

In [4]:
df.head(6)

,AAPL,AMZN,GOOG,MSFT,NFLX
Date,,,,,
2010-01-04,6.515214,6.6950,15.610239,23.683069,7.640000
2010-01-05,6.526476,6.7345,15.541497,23.690723,7.358571
2010-01-06,6.422663,6.6125,15.149715,23.545328,7.617143
2010-01-07,6.410790,6.5000,14.797037,23.300467,7.485714
2010-01-08,6.453412,6.6760,14.994298,23.461164,7.614286
2010-01-11,6.396483,6.5155,14.971633,23.162731,7.604286


Wir werden im Folgenden mehrere Hilfsfunktionen definieren: 
- Die Funktion `daily_weights` berechnet mit Hilfe von `scipy.minimize` Funktionalität auf Tagesebene die optimalen Portfoliogewichte und implementiert den Ansatz eines rollierenden Zeitfensters über eine for-Schleife. Das Argument *window* legt die Länge des Zeitfensters fest.
- Zwei sogenannte `lambda` Funktionen zur Berechnung kumulativer Renditen und der täglichen Sharpe-Ratio. Lambda Funktionen sind anonyme Funktionen, die nur aus einer Zeile Code bestehen.
- Die Hauptfunktion `strat_sr` berechnet die annualisierte Sharpe-Ratio der Strategie. Das Argument *hold* legt die Länge der Halteperiode fest.

Zunächst schreiben wir die Funktion zur Berechnung der Portfoliostandardabweichung und die Zielfunktion `min_variance` zur Bestimmung der Gewichte des Minimum Varianz Portfolios mit Hilfe der numerischen Optimierung. Beide Funktionen wurden im **Notebook C.2** eingeführt. 

In [5]:
# helper functions to calculate the weights of the minimum variance
# portfolio (see notebook C.2)
def calc_portfolio_std(weights, cov):
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov, weights))) * np.sqrt(252)
    return portfolio_std

def min_variance(cov):
    num_assets = len(cov)
    args = cov
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (0.0,1.0)
    bounds = tuple(bound for asset in range(num_assets))
    result = sco.minimize(calc_portfolio_std, num_assets*[1./num_assets,], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

Und nun die oben erwähnten Hilfsfunktionen:

In [6]:
# helper functions: weights, compound returns, sharpe ratio
def daily_weights(df, window=100):
    empty_list = list()
    for iStart in range(0, len(df)-window):        
        iEnd = iStart+window        
        # Calculate annualized sample covariance matrix
        Sigma = df[iStart:iEnd].pct_change().cov()*252
        # Calculate weights of minimum variance portfolio
        w = min_variance(Sigma)
        empty_list.append(w)
                
    weights0 = pd.DataFrame(empty_list, columns=df.columns)
    df_new = df[window:len(df)].reset_index('Date')
    weights = pd.concat([df_new['Date'], weights0], axis=1)
    weights = weights.set_index('Date')               
    return weights

# cumulative returns
compound = lambda x: (1 + x).prod() - 1

# daily sharpe ratio
daily_sr = lambda x: x.mean() / x.std()

# strategie returns (main function)
def strat_sr(prices, hold, window=100):
    # weights = port
    freq = '%dB' % hold # holding period in number of business days
    port = daily_weights(prices) # daily security weights 
    
    # daily returns
    daily_rets = prices.pct_change()
    daily_rets = daily_rets[window:len(prices)]
    
    # calculation of portfolio returns
    port = port.shift(1).resample(freq).first() # time series with 'freq' as frequency
    returns = daily_rets.resample(freq).apply(compound)
    port_rets = (port * returns).sum(axis=1) # sum of weighted security returns
    
    return daily_sr(port_rets) * np.sqrt(252/hold)
    

Führen wir beispielhaft die Strategie mit einer Halteperiode von 20 Handelstagen aus. D.h., alle 20 Tage erfolgt ein Rebalancing des Portfolios an die dann jeweils optimalen Gewichte.

In [7]:
strat_sr(df,20)

1.2219773614056912